In [48]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [49]:
import grader

# Mapreduce

## Introduciton

We are going to be running mapreduce jobs on the wikipedia dataset.  The dataset is available (pre-chunked) on [s3](s3://dataincubator-course/mrdata/simple/).

For development, you can even use a single chunk (eg. part-00026.xml.bz2). That is small enough that mrjob can process the chunk in a few seconds. Your development cycle should be:

1.  Get your job to work locally on one chunk.  This will greatly speed up your
development.  To run on local:
```bash
python job_file.py -r local data/wikipedia/simple/part-00026.xml.bz2 > /tmp/output.txt
```
    
2.  Get your job to work on the full dataset on GCP (Google Cloud Platform).  This will greatly speed up your production.  To run on GCP ([details](https://pythonhosted.org/mrjob/guides/dataproc-quickstart.html)):
```bash
python job_file.py -r dataproc data/wikipedia/simple/part-00026.xml.bz2 \
    --output-dir=gs://my-bucket/output/ \
    --no-output 
```

    Not that you can also pass an entire local directory of data (eg. `data/simple/`) as the input.

### Note on Memory
There's a large difference between developing locally on one chunk and running your job on the entire dataset.  While you can get away with sloppy memory use locally, you really need to keep memory usage down if you hope to be able to complete the miniproject.  Remember, memory needs to be $O(1)$, not $O(n)$ in input.

### Multiple Mapreduces
You can combine multiple steps by overriding the [steps method](https://pythonhosted.org/mrjob/guides/writing-mrjobs.html#multi-step-jobs).  Usually your mapreduce might look like this
```python
from mrjob.job import MRJob

class SingleMRJob(MRJob):
    def mapper(self, key, value):
        pass

    def reducer(self, key, values):
        pass
```

`MRJob` automatically uses the `mapper` and `reducer` methods.  To specify multiple steps, you need to override the `steps` method:

```python
from mrjob.job import MRJob
from mrjob.step import MRStep

class MultipleMRJob(MRJob):
    def mapper1(self, key, value):
        pass

    def reducer1(self, key, values):
        pass
        
    def mapper2(self, key, value):
        pass

    def reducer2(self, key, values):
        pass
        
    def steps(self):
        return [
            MRStep(mapper=self.mapper1, reducer=self.reducer1),
            MRStep(mapper=self.mapper2, reducer=self.reducer2),
        ]
```

As a matter of good style, we recommend that you actually write each individual mapreduce as it's own class.  Then write a wrapper module whose sole job is to combine those mapreduces by overriding `steps`.

Some simple boilerplate for this, taking advantage of the default `steps` function that we get for free in a single-step MRJob class:

```python
class FirstStep(MRJob):
  def mapper(self, key, value):
    pass
  def reducer(self, key, values):
    pass
  
class SecondStep(MRJob):
  def mapper(self, key, value):
    pass
  def reducer(self, key, values):
    pass
  
class SteppedJob(MRJob):
  """
  A two-step job that first runs FirstStep's MR and then SecondStep's MR
  """
  def steps(self):
    return FirstStep().steps() + SecondStep().steps()
```


### Note on Style
Here are some helpful articles on how mrjob works and how to pass parameters to your script:
  - [How mrjob is run](https://pythonhosted.org/mrjob/guides/concepts.html#how-your-program-is-run)
  - [Adding passthrough options](https://pythonhosted.org/mrjob/job.html#mrjob.job.MRJob.add_passthrough_option)
  - [An example of someone solving similar problems](http://arunxjacob.blogspot.com/2013/11/hadoop-streaming-with-mrjob.html)

See the notebook "Hadoop MapReduce with mrjob" in the datacourse for more details.

Finally, if you are find yourself processing a lot of special cases, you are probably doing it wrong.  For example, mapreduce jobs for `Top100WordsSimpleWikipediaPlain`, `Top100WordsSimpleWikipediaText`, and `Top100WordsSimpleWikipediaNoMetaData` are less than 150 lines of code (including generous blank lines and biolerplate code).

## Question 1: top100_words_simple_plain
Return a list of the top 100 words in an article text (in no particular order). You will need to write this as two map reduces:

1. The first job is similar to standard wordcount but with a few tweaks. The data provided for wikipedia is in `*.xml.bz2` format.  Mrjob will automatically decompress `bz2`.  We'll deal with the `xml` in the next question. For now, just treat it as text.  A few hints:
   - To split the words, use the regular expression "\w+".
   - Words are not case sensitive: i.e. "The" and "the" reference to the same word.  You can use `string.lower()` to get a single case-insenstive canonical version of the data.

2. The second job will take a collection of pairs `(word, count)` and filter for only the highest 100.  A few notes:
    - **Passing parameters:** To make the job more reusable make the job find the largest `n` words where `n` is a parameter obtained via [`get_jobconf_value`](https://pythonhosted.org/mrjob/utils-compat.html).
    - **Keeping track of the top n:** We have to keep track of at most the `n` most popular words.  As long as `n` is small, e.g. 100, we can keep track of the *running largest n* in memory wtih a priority-queue. We suggest taking a look at `heapq` ([details](https://docs.python.org/2/library/heapq.html)), part of the Python standard library for this.  It allows you to push elemnets into a list while keeping track of the highest priority element.
```python
h = []
heappush(h, (5, 'write code'))
heappush(h, (7, 'release product'))
heappush(h, (1, 'write spec'))
heappush(h, (3, 'create tests'))
heappop(h)  // returns (1, 'write spec')
```
   
       A naive implementation would cost $O(1)$ to insert but $O(n)$ to retrieve.  `heapq` uses a [self-balancing binary search tree](https://en.wikipedia.org/wiki/Self-balancing_binary_search_tree) to enable $O(\log(n))$ insertion and $O(1)$ removal. You may be asked about this data structure on an interview so it is good to get practice with it now.
    - **Working across nodes:** To obtain the largest `n`, we need to first obtain the largest n elements per chunk from the mapper, output them to the same key (reducer), and then collect the largest n elements of those in the reducer (**Question:** why does this gaurantee that we have found the largest n over the entire set?)
    - **Working within a node:** Given that we are using a priority queue, we will need to first initialize it, then `push` or `pushpop` each record to it, and finally output the top `n` after seeing each record.  For mappers, notice that these three phases correspond nicely to these three functions:
        - `mapper_init`
        - `mapper`
        - `mapper_final`

    There are similar functions in the reducer.  Also, while the run method to launch the mapreduce job is a classmethod:
        ```python
          if __name__ == '__main__':
            MRWordCount.run()
        ```
     actual instances of our mapreduce are instantiated on the map and reduce nodes.  More precisely, a separate mapper class is instantiated in each map node and a reducer class is instantiated in each reducer node.  This means that the three mapper functions can pass state through `self`, e.g. `self.heap`. Remember that to pass state between the map and reduce phase, you will have to use `yield` in the mapper and read each line in the reducer. (**Question:** Can you pass state between two mappers?)

**Checkpoint:**
- Total unique words: 1,584,646

In [2]:
import pandas as pd


In [11]:
result = pd.read_csv('output2.txt', sep = '\t', header = None)

In [12]:
result

,0,1
0,-1572530,the
1,-1400092,quot
2,-1211881,gt
3,-1205636,lt
4,-1142726,id
5,-972021,of
6,-658172,in
7,-633840,and
8,-604105,text
9,-576314,a


In [14]:
result.ix[0][0]
result2 = []
for i in range(0,100):
    result2.append((result.ix[i][1], -result.ix[i][0]))


In [15]:
def top100_words_simple_plain():
    return result2

grader.score(question_name='mr__top100_words_simple_plain', func=top100_words_simple_plain)

Your score:  0.96


## Question 2: top100_words_simple_text
Notice that the words "page" and "text" make it into the top 100 words in the previous problem.  These are not common English words!  If you look at the xml formatting, you'll realize that these are xml tags.  You should parse the files so that tags like `<page></page>` should not be included in your total, nor should words outside of the tag `<text></text>`.

**Hints**:
1. Both `xml.etree.elementtree` from the Python stdlib or `lxml.etree` parse xml. `lxml` is significantly faster though and avoids some bugs.

2. In order to parse the text, we will have to accumulate a `<page></page>` worth of data and then split the resulting string into words.

3. Don't forget that the Wikipedia format can have multiple revisions but you only want the latest one.

4. What happens if a content from a page is split across two different mappers? How does this problem scale with data size?

**Checkpoint:**
- Total unique words: 867,871

In [6]:
import pandas as pd
result = pd.read_csv('output-04-13-2.txt', sep = '\t', header = None)
resultQ2 = []
for i in range(0,100):
    resultQ2.append((result.ix[i][1], -result.ix[i][0]))
resultQ2

[('the', 1556257),
 ('of', 947319),
 ('in', 646320),
 ('and', 619414),
 ('a', 568273),
 ('to', 445053),
 ('is', 404778),
 ('ref', 369992),
 ('category', 325444),
 ("''", 301153),
 ('1', 230879),
 ('http', 216835),
 ('was', 212856),
 ('0', 211904),
 ('for', 209075),
 ('it', 206932),
 ('2', 194447),
 ('on', 185674),
 ('name', 176581),
 ('br', 166079),
 ('as', 156504),
 ('www', 156113),
 ('that', 155920),
 ('font', 153442),
 ('s', 151839),
 ('align', 148611),
 ('by', 148147),
 ('user', 146052),
 ('from', 143521),
 ('style', 136039),
 ('are', 133702),
 ('with', 131420),
 ('he', 124840),
 ('com', 122102),
 ('this', 121297),
 ('color', 120111),
 ('3', 116429),
 ('title', 115694),
 ('at', 111461),
 ('center', 110964),
 ('talk', 109692),
 ('be', 108497),
 ("'''", 106161),
 ('or', 105190),
 ('small', 102497),
 ('i', 101595),
 ('an', 99276),
 ('not', 92084),
 ('new', 89331),
 ('nbsp', 89040),
 ('football', 88718),
 ('people', 86907),
 ('4', 86892),
 ('b', 83304),
 ('united', 82711),
 ('cite', 82

In [10]:
def top100_words_simple_text():
    return resultQ2

grader.score(question_name='mr__top100_words_simple_text', func=top100_words_simple_text)

Your score:  0.94


## Question 3: top100_words_simple_no_metadata

Finally, notice that 'www' and 'http' make it into the list of top 100 words in the previous problem.  These are also not common English words either!  These are clearly from the url in hyperlinks.  Looking at the format of [Wikipedia links](http://en.wikipedia.org/wiki/Help:Wiki_markup#Links_and_URLs) and [citations](http://en.wikipedia.org/wiki/Help:Wiki_markup#References_and_citing_sources), you'll notice that they tend to appear within single and double brackets and curly braces.

**Hint**:
You can either write a simple parser to eliminate the urls within brackets, angle braces, and curly braces or you can use a package like the colorfully-named [mwparserfromhell](https://github.com/earwig/mwparserfromhell/), which has been provisioned on `mrjob` and supports the convenient helper function `strip_code()` (which is used by the reference solution).

**Checkpoint:**
- Total unique words: 618,410

In [17]:
import pandas as pd
result = pd.read_csv('output-04-13-Q3-2.txt', sep = '\t', header = None)
resultQ3 = []
for i in range(0,100):
    resultQ3.append((result.ix[i][1], -result.ix[i][0]))
resultQ3

[('the', 1436163),
 ('of', 747501),
 ('in', 587950),
 ('and', 547783),
 ('a', 518416),
 ('to', 417344),
 ('is', 391185),
 ('category', 279294),
 ('was', 209439),
 ('it', 199374),
 ('for', 185318),
 ('on', 163328),
 ('0', 160764),
 ('that', 152104),
 ('as', 148990),
 ('align', 141185),
 ('by', 132371),
 ('are', 129042),
 ('1', 127618),
 ('from', 126571),
 ('2', 126202),
 ('he', 122310),
 ('with', 121317),
 ('this', 109495),
 ('be', 101466),
 ('or', 95290),
 ('at', 94603),
 ('an', 93926),
 ('center', 91753),
 ('i', 87166),
 ('not', 86750),
 ('style', 79691),
 ('3', 78385),
 ('people', 77728),
 ('other', 76241),
 ('they', 74552),
 ('his', 73024),
 ('have', 66933),
 ('utc', 65872),
 ('has', 63436),
 ('also', 60638),
 ('talk', 60428),
 ('american', 59590),
 ('bgcolor', 59077),
 ('one', 58577),
 ('right', 58505),
 ('4', 58280),
 ('which', 54597),
 ('but', 53623),
 ('were', 52489),
 ('can', 51774),
 ('5', 51175),
 ('new', 51076),
 ('first', 50762),
 ('there', 49557),
 ('b', 45524),
 ('referen

In [18]:
def top100_words_simple_no_metadata():
    return (resultQ3)

grader.score(question_name='mr__top100_words_simple_no_metadata', func=top100_words_simple_no_metadata)

Your score:  0.93


## Question 4: wikipedia_entropy
The [Shannon entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory) of a discrete random variable with probability mass function $p(x)$ is:

$$H(X) = - \sum p(x) \log_2 p(x)$$

You can think of the Shannon entropy as the number of bits needed to represent the random variable if it were optimally compressed.  It is also closely tied to the notion of entropy from physics.

You'll be estimating the Shannon entropy of different Simple English and Thai based off of their Wikipedias. Do this with n-grams of characters, by first calculating the entropy of a single n-gram and then dividing by n to get the per-character entropy. Use n-grams of size 1, 2, 3.  How should our per-character entropy estimates vary with n?  How should they vary by
the size of the corpus? How much data would we need to get reasonable entropy estimates for each n?

The data you need is available at:
    - https://s3.amazonaws.com/dataincubator-course/mrdata/simple/part-000\*
    - https://s3.amazonaws.com/dataincubator-course/mrdata/thai/part-000\*

*Question*: Why do we need to use map-reduce? There are >300 million characters in this dataset. How much memory would it take to store all `n`-grams as `n` increases?

Notes:
1. Characters are case sensitive.
1. Do not use the previous regex `\w+` to split --- depending on your system configuration, this may only match English characters, which would severely skew entropy estimates for Thai. Be careful about unicode.
1. Please treat all whitespace as the same character.  You can do this by
  `" ".join(text.split())`
1. For reference, the exact code we use to extract text is:

```
    wikicode = mwparserfromhell.parse(text)
    text = " ".join(" ".join(fragment.value.split())
                    for fragment in wikicode.filter_text())
```

A naive implementation of this job will take a very long time to run.  Instead, we will need to use a few optimizations:
1. See [this post](http://www.johndcook.com/blog/2013/08/17/calculating-entropy/) on how to calculate entropy efficiently.
2. It turns out that writing to disk is the most expensive part of a map-reduce.  [Zipf's law](https://en.wikipedia.org/wiki/Zipf's_law) tells us that only a handful (relatively-speaking) of n-grams make up most of our observations.  Can you do a map-side cache of these values to reduce the number of writes?
   
   Note that it may not be possible to match the reference solution by omitting n-grams.
3. Entropy is a function of the count distribution, i.e. it is independent of which ngrams correspond to which counts.  If we have N singleton ngrams, it's more efficient to (somehow) encode that as "N singleton ngrams" rather than as N key-value pairs:
```
    (word1, 1)
    (word2, 1)
    (word3, 1)
    ...
```
   Can you use a in-memory cache to solve this problem?  What fraction of ngrams only occur once?  How much of a speedup do you expect to get from this optimization?

In [7]:
def wikipedia_entropy():
    return [
        ("simple1", 5.0529307635),
        ("simple2", 4.5517101734),
        ("simple3", 4.1409467817),
        ("thai1", 6.243533367),
        ("thai2", 5.3387320524),
        ("thai3", 4.6824655198),
    ]

grader.score(question_name='mr__wikipedia_entropy', func=wikipedia_entropy)

Your score:  1.0


## Question 5: link_stats_simple
Let's look at some summary statistics on the number of unique links on a page to other Wikipedia articles.  Return the number of articles (count), average number of links, standard deviation, and the 25%, median, and 75% quantiles.

1. Notice that the library `mwparserfromhell` supports the method `filter_wikilinks()`.
2. You will need to compute these statistics in a way that requires O(1) memory.  You should be able to compute the first few (i.e. non-quantile) statistics exactly by looking at the first few moments of a distribution. The quantile quantities can be accurately estimated by using reservoir sampling with a large reservoir.
3. If there are multiple links to the article have it only count for 1.  This keeps our results from becoming too skewed.
4. Don't forget that some (a surprisingly large number of) links have unicode! Make sure you treat them correctly.

In [3]:
def link_stats_simple():
    return [
        ("count", 188408.0),
        ("mean", 15.6400949004),
        ("stdev", 50.6331804249),  ## 582
        ("25%", 1.0), ##1
        ("median", 6.0), ## 9
        ("75%", 17.0), ##80
    ]

grader.score(question_name='mr__link_stats_simple', func=link_stats_simple)

Your score:  1.0


## Question 6: link_stats_english
The same thing but for all of English Wikipedia.  This is the real test of how well your algorithm scales!  The data is also located on [s3](s3://dataincubator-course/mrdata/english/).

**Note:**
Because of the size of the dataset, this job may take several hours to complete. It's advisable to run it overnight once you're reasonably sure it will work (due to testing the code on smaller inputs).

As a barometer, our reference solution takes around 5 hours to run.

In [13]:
def link_stats_english():
    return [
        ("count", 15017088),
        ("mean", 14.0977769495),
        ("stdev", 62.4357253132),
        ("25%", 1.),
        ("median", 1.),
        ("75%", 14.),
    ]

grader.score(question_name='mr__link_stats_english', func=link_stats_english)

Your score:  1.0


## Question 7: double_link_stats_simple
Instead of analyzing single links, let's look at double links.  That is, pages `A` and `C` that are connected through many pages `B` where there is a link `A -> B -> C` or `C -> B -> A`. Find the list of all pairs `(A, C)` (you can use alphabetical ordering to break symmetry) that have the 100 "most" connections (see below for the definition of "most").  This should give us a notion that the articles `A` and `C` refer to tightly related concepts.

1. This can be thought of as a Matrix Multiplication problem.  If the adjacency matrix is denoted $M$ (where $M_{ij}$ represents the link between $i$ an $j$), we are looking for the highest 100 elements of the matrix $M M$. Note that this doesn't mean constructing matrices is the most efficent way to solve the problem.

2. Notice that a lot of Category pages (denoted "Category:.*") have a high link count and will rank very highly according to this metric.  Wikipedia also has `Talk:` pages, `Help:` pages, and static resource `Files:`.  All such "non-content" pages (and there might be more than just this) and links to them should be first filtered out in this analysis.

3. Some pages have more links than others.  If we just counted the number of double links between pages, we will end up seeing a list of articles with many links, rather than concepts that are tightly connected.

   1. One strategy is to weight each link as $\frac{1}{n}$ where $n$ is the number links on the page.  This way, an article has to spread it's "influence" over all $n$ of its links.  However, this can throw off the results if $n$ is small.

   2. Instead, try using [Bayesian Smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) by weighting each link as $\frac{1}{n+10}$ where 10 sets the "scale" in terms of number of links above which a page becomes "significant".  The number 10 was somewhat arbitrarily chosen but seems to give reasonably relevant results.

   3. This means that our "count" for a pair A,C will be the products of the two link weights between them, summed up over all their shared connections.

4. Again, if there are multiple links from a page to another, have it only count for 1.  This keeps our results from becoming skewed by a single page that references the same page multiple times.

5. You'll need to filter out the cases where A -> B -> A, but just note that the reference solution does this after calculating the link weights.

6. The links and page titles are encoded into utf-8.

Don't be afraid if these answers are not particularly insightful.  Simple Wikipedia is not as rich as English Wikipedia.  However, you should notice that the articles are closely related conceptually.


In [47]:
import pandas as pd
da = pd.read_csv('what_is_this_answer.txt', header = None, sep='\t')
da.columns = ('k', 'v')
da

answer = []
for i in range(0,100):
    l1 = da.k.str.strip('[]')[i].split(',')[0].strip('""')
    l2 = da.k.str.strip('[]')[i].split(',')[1].strip('""')
    v = da.v[i]
    answer.append(((l1,l2),v))
answer[77]

(('japan national football team', 'list of japan international footballers'),
 0.053626040599999994)

,k,v
0,"(list of cities in idaho, united states)",0.056029
1,"(communes of the alpes-maritimes department, f...",0.056114
2,"(communes of france, communes of the vaucluse ...",0.057264
3,"(city, list of cities in oklahoma)",0.057611
4,"(idaho, list of cities in idaho)",0.056152
5,"(ain, communes of the ain department)",0.058061
6,"(communes of the vaucluse department, vaucluse)",0.057575
7,"(list of settlements in latvia, town rights)",0.058073
8,"(communes of the vaucluse department, provence...",0.057638
9,"(communes of the alpes-maritimes department, p...",0.057565


In [50]:
def double_link_stats():
    return answer

grader.score(question_name='mr__double_link', func=double_link_stats)

Your score:  0.9


*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*